In [1]:
import os
import subprocess

DATA_DIR = "data"
if not os.path.exists(DATA_DIR):
    os.mkdir(DATA_DIR)


for month in range(1, 13):
    if not os.path.exists(f"{DATA_DIR}/{month}.parquet"):
        subprocess.run(
            [
                "wget",
                f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-{month:02}.parquet",
                "-O",
                f"{DATA_DIR}/{month}.parquet",
            ]
        )


total_size = sum(
    os.path.getsize(f"{DATA_DIR}/{month}.parquet") for month in range(1, 13)
)  # bytes
total_size_mb = total_size // (1024 * 1024)
print(f"Total dataset size: {total_size_mb} MB")

Total dataset size: 660 MB


In [2]:
if not os.path.exists(f"{DATA_DIR}/taxi_zone_lookup.csv"):
    subprocess.run(
        [
            "wget",
            "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv",
            "-O",
            f"{DATA_DIR}/taxi_zone_lookup.csv",
        ]
    )

In [3]:
import polars as pl

In [4]:
dfs = []
for month in range(1, 13):
    df = pl.scan_parquet(f"{DATA_DIR}/{month}.parquet")
    df = df.with_columns(
        pl.col("tpep_pickup_datetime").dt.cast_time_unit("ms"),
        pl.col("tpep_dropoff_datetime").dt.cast_time_unit("ms"),
    )
    df = df.filter(
        pl.col("tpep_pickup_datetime").dt.year() >= 2024,
        pl.col("tpep_dropoff_datetime").dt.date() <= pl.date(2025, 1, 1)
    )
    df = df.with_columns(pl.col("passenger_count").cast(pl.UInt8))
    df = df.with_columns(pl.col("RatecodeID").cast(pl.UInt8))
    df = df.with_columns(pl.col("VendorID").cast(pl.UInt8))
    df = df.with_columns(pl.col("PULocationID").cast(pl.UInt16))
    df = df.with_columns(pl.col("DOLocationID").cast(pl.UInt16))

    # df = df.with_columns(pl.col("store_and_fwd_flag").cast(pl.Categorical))
    dfs.append(df)

df_2024 = pl.concat(dfs)

In [5]:
df_2024.collect()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
u8,datetime[ms],datetime[ms],u8,f64,u8,str,u16,u16,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,2024-01-01 00:57:55,2024-01-01 01:17:43,1,1.72,1,"""N""",186,79,2,17.7,1.0,0.5,0.0,0.0,1.0,22.7,2.5,0.0
1,2024-01-01 00:03:00,2024-01-01 00:09:36,1,1.8,1,"""N""",140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
1,2024-01-01 00:17:06,2024-01-01 00:35:01,1,4.7,1,"""N""",236,79,1,23.3,3.5,0.5,3.0,0.0,1.0,31.3,2.5,0.0
1,2024-01-01 00:36:38,2024-01-01 00:44:56,1,1.4,1,"""N""",79,211,1,10.0,3.5,0.5,2.0,0.0,1.0,17.0,2.5,0.0
1,2024-01-01 00:46:51,2024-01-01 00:52:57,1,0.8,1,"""N""",211,148,1,7.9,3.5,0.5,3.2,0.0,1.0,16.1,2.5,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2,2024-12-31 23:32:00,2024-12-31 23:56:00,null,10.71,null,null,16,7,0,-4.13,0.0,0.5,0.0,0.0,1.0,-2.63,null,null
2,2024-12-31 23:05:00,2024-12-31 23:18:00,null,4.56,null,null,252,16,0,-1.68,0.0,0.5,0.0,0.0,1.0,-0.18,null,null
2,2024-12-31 23:03:16,2024-12-31 23:28:35,null,3.94,null,null,181,255,0,4.46,0.0,0.5,5.19,0.0,1.0,11.15,null,null


I cant cast to Categorical. I am getting error. Why?

In [6]:
df_taxi_zones = pl.scan_csv("data/taxi_zone_lookup.csv")
df_taxi_zones.collect()

LocationID,Borough,Zone,service_zone
i64,str,str,str
1,"""EWR""","""Newark Airport""","""EWR"""
2,"""Queens""","""Jamaica Bay""","""Boro Zone"""
3,"""Bronx""","""Allerton/Pelham Gardens""","""Boro Zone"""
4,"""Manhattan""","""Alphabet City""","""Yellow Zone"""
5,"""Staten Island""","""Arden Heights""","""Boro Zone"""
…,…,…,…
261,"""Manhattan""","""World Trade Center""","""Yellow Zone"""
262,"""Manhattan""","""Yorkville East""","""Yellow Zone"""
263,"""Manhattan""","""Yorkville West""","""Yellow Zone"""


# 2

In [7]:
trip_time = (pl.col("tpep_dropoff_datetime") - pl.col("tpep_pickup_datetime")).alias(
    "trip_time"
)
money_cols = ["fare_amount", "extra", "mta_tax", "tip_amount", "tolls_amount", "improvement_surcharge", "total_amount", "congestion_surcharge", "Airport_fee"]

In [8]:
df_2024 = (
    df_2024.with_columns(
        pl.col("passenger_count").fill_null(1)
    )
    .filter(pl.col("passenger_count") > 0)
    .with_columns(
        pl.col("passenger_count").clip(upper_bound=6)
    )
    .filter(trip_time.dt.total_minutes() < 120)
    .with_columns(
        [
            pl.col(name).abs()
            for name in money_cols
        ]
    )
    .filter(
        [
            pl.col(name) < 1000
            for name in money_cols
        ]
    )
    .filter(
        pl.col("RatecodeID").is_not_null(),
        pl.col("RatecodeID") != 99,
        pl.col("VendorID").is_not_null(),
        pl.col("VendorID").is_in([1, 2, 6, 7])
    )
    # .collect()
)

# 3

In [9]:
other_payments = [0, 3, 4, 5, 6]  # I added Flex Fare trip and will set 0 as "other"
week_days = list(range(1, 6))
weekend_days = [6, 7]

In [10]:
df_2024.collect().head()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
u8,datetime[ms],datetime[ms],u8,f64,u8,str,u16,u16,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,2024-01-01 00:57:55,2024-01-01 01:17:43,1,1.72,1,"""N""",186,79,2,17.7,1.0,0.5,0.0,0.0,1.0,22.7,2.5,0.0
1,2024-01-01 00:03:00,2024-01-01 00:09:36,1,1.8,1,"""N""",140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
1,2024-01-01 00:17:06,2024-01-01 00:35:01,1,4.7,1,"""N""",236,79,1,23.3,3.5,0.5,3.0,0.0,1.0,31.3,2.5,0.0
1,2024-01-01 00:36:38,2024-01-01 00:44:56,1,1.4,1,"""N""",79,211,1,10.0,3.5,0.5,2.0,0.0,1.0,17.0,2.5,0.0
1,2024-01-01 00:46:51,2024-01-01 00:52:57,1,0.8,1,"""N""",211,148,1,7.9,3.5,0.5,3.2,0.0,1.0,16.1,2.5,0.0


In [13]:
df_2024 = (
    df_2024.with_columns(
        pl.when(pl.col("payment_type").is_in(other_payments)).then(pl.lit(0)).otherwise(pl.col("payment_type")).alias("payment_type")
    )
    .join(
        df_taxi_zones, left_on="PULocationID", right_on="LocationID"
    )
    .drop(["PULocationID", "Zone", "service_zone"])
    .rename({"Borough": "PULocation_Borough"})
    .join(
        df_taxi_zones, left_on="DOLocationID", right_on="LocationID"
    )
    .drop(["DOLocationID", "Zone", "service_zone"])
    .rename({"Borough": "DOLocationID_Borough"})
    .with_columns(
        (pl.col("Airport_fee") > 0).alias("is_airport_ride"),
        pl.when(
            pl.col("tpep_pickup_datetime").dt.weekday().is_in(week_days)
            &
            (
                (
                    (pl.col("tpep_pickup_datetime").dt.time() > pl.time(hour=6, minute=30))
                    &
                    (pl.col("tpep_pickup_datetime").dt.time() < pl.time(hour=9, minute=30))
                )
                |
                (
                    (pl.col("tpep_pickup_datetime").dt.time() > pl.time(hour=15, minute=30))
                    &
                    (pl.col("tpep_pickup_datetime").dt.time() < pl.time(hour=20, minute=0))
                )
            )
        )
        .then(pl.lit(True))
        .otherwise(pl.lit(False))
        .alias("is_rush_hour")
    )
    # Is there a way to make it look nicer? I got wylew looking at those AND OR XD
)

# Is there a way to make it look nicer? I got wylew looking at those AND OR XD

# 4

In [15]:
df_2024_eager = df_2024.collect()

In [17]:
df_2024_eager

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,PULocation_Borough,DOLocationID_Borough,is_airport_ride,is_rush_hour
u8,datetime[ms],datetime[ms],u8,f64,u8,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,bool,bool
2,2024-01-01 00:57:55,2024-01-01 01:17:43,1,1.72,1,"""N""",2,17.7,1.0,0.5,0.0,0.0,1.0,22.7,2.5,0.0,"""Manhattan""","""Manhattan""",false,false
1,2024-01-01 00:03:00,2024-01-01 00:09:36,1,1.8,1,"""N""",1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0,"""Manhattan""","""Manhattan""",false,false
1,2024-01-01 00:17:06,2024-01-01 00:35:01,1,4.7,1,"""N""",1,23.3,3.5,0.5,3.0,0.0,1.0,31.3,2.5,0.0,"""Manhattan""","""Manhattan""",false,false
1,2024-01-01 00:36:38,2024-01-01 00:44:56,1,1.4,1,"""N""",1,10.0,3.5,0.5,2.0,0.0,1.0,17.0,2.5,0.0,"""Manhattan""","""Manhattan""",false,false
1,2024-01-01 00:46:51,2024-01-01 00:52:57,1,0.8,1,"""N""",1,7.9,3.5,0.5,3.2,0.0,1.0,16.1,2.5,0.0,"""Manhattan""","""Manhattan""",false,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2,2024-12-31 22:56:33,2024-12-31 23:05:57,1,1.9,1,"""N""",1,12.1,1.0,0.5,1.88,0.0,1.0,18.98,2.5,0.0,"""Manhattan""","""Manhattan""",false,false
2,2024-12-31 23:18:37,2024-12-31 23:32:32,1,3.88,1,"""N""",2,19.1,1.0,0.5,0.0,0.0,1.0,24.1,2.5,0.0,"""Manhattan""","""Queens""",false,false
2,2024-12-31 23:08:52,2024-12-31 23:34:59,1,5.53,1,"""N""",1,29.6,1.0,0.5,2.0,0.0,1.0,36.6,2.5,0.0,"""Manhattan""","""Brooklyn""",false,false


In [24]:
dataset = (
    df_2024_eager.to_dummies(
        ["PULocation_Borough", "DOLocationID_Borough"]  # I skip payment_type as it is int with 3 values (maybe I misunderstood smt earlier)
    )
    .with_columns(
        pl.col("passenger_count").count()
        .over(pl.col("tpep_pickup_datetime").dt.date())
        .alias("total_number_of_rides"),

        pl.col("is_airport_ride").sum()
        .over(pl.col("tpep_pickup_datetime").dt.date())
        .alias("number_of_airport_rides"),

        pl.col("is_rush_hour").sum()
        .over(pl.col("tpep_pickup_datetime").dt.date())
        .alias("number_of_rush_hour_rides"),

        pl.col("fare_amount").mean()
        .over(pl.col("tpep_pickup_datetime").dt.date())
        .alias("avg_fare_amount"),

        pl.col("trip_distance").median()
        .over(pl.col("tpep_pickup_datetime").dt.date())
        .alias("median_distance"),

        pl.col("total_amount").sum()
        .over(pl.col("tpep_pickup_datetime").dt.date())
        .alias("sum_of_total_amounts"),

        pl.col("total_amount").filter(pl.col("payment_type") == 0).sum()
        .over(pl.col("tpep_pickup_datetime").dt.date())
        .alias("sum_of_total_amounts_other"),

        pl.col("total_amount").filter(pl.col("payment_type") == 1).sum()
        .over(pl.col("tpep_pickup_datetime").dt.date())
        .alias("sum_of_total_amounts_card"),

        pl.col("total_amount").filter(pl.col("payment_type") == 2).sum()
        .over(pl.col("tpep_pickup_datetime").dt.date())
        .alias("sum_of_total_amounts_cash"),

        pl.col("congestion_surcharge").sum()
        .over(pl.col("tpep_pickup_datetime").dt.date())
        .alias("total_congestion_surcharge"),

        pl.col("passenger_count").sum()
        .over(pl.col("tpep_pickup_datetime").dt.date())
        .alias("total_passenger_count"),

        pl.col("tpep_pickup_datetime").dt.quarter()
        .alias("quarter"),

        pl.col("tpep_pickup_datetime").dt.month()
        .alias("month"),

        pl.col("tpep_pickup_datetime").dt.day()
        .alias("day"),

        pl.col("tpep_pickup_datetime").dt.weekday()
        .alias("day_of_week"),

        pl.col("tpep_pickup_datetime").dt.weekday().is_in(weekend_days)
        .alias("is_weekend"),

        pl.col("tpep_pickup_datetime").dt.date()
        .alias("date"),
    )
    .drop([
        "VendorID", "tpep_pickup_datetime", "tpep_dropoff_datetime", "RatecodeID", "store_and_fwd_flag", "extra",
        "mta_tax", "tip_amount", "tolls_amount","improvement_surcharge", "congestion_surcharge", "Airport_fee"
    ])
)


In [26]:
dataset.write_parquet("data/dataset.parquet")

# 5

![Alt Text](sleep.png)